# jammer
## 1 Extract, transform, and load
### I- Data for Starfish

Michael Gully-Santiago  
Friday, March 31, 2017  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [2]:
from astropy.io import fits
import h5py

In [3]:
import astropy.units as u

In [4]:
from os import listdir

In [5]:
files = listdir('../data/raw/')

We need to convert the units to what Starfish expects:
- **wavelengths:** Angstroms
- **fluxes:** erg/s/cm$^2$/A

update 4/21/2017:
Some sigmas have negative or zero values!!!  Fix this!

In [16]:
for i, file in enumerate(files):
    with open('../data/raw/{}'.format(file), 'rb') as f:
        wlgrid, Flux, Flux_err = np.load(f, allow_pickle=True, encoding='bytes')
        
    out_name = '../data/reduced/{}.hdf5'.format(file[:-4])
    fls_out = (Flux*u.Watt/u.m**2/u.m).to(u.erg/u.s/u.cm**2/u.Angstrom).value
    sig_out = (Flux_err*u.Watt/u.m**2/u.m).to(u.erg/u.s/u.cm**2/u.Angstrom).value
    #print(out_name, np.min(sig_out), np.sum(sig_out==0), np.percentile(fls_out/sig_out, 80))
    bi = sig_out <= 0
    sig_out[bi] = np.abs(fls_out[bi])
    wls_out = wlgrid*10000.0
    msk_out = np.ones(len(wls_out), dtype=int)
    f_new = h5py.File(out_name, 'w')
    f_new.create_dataset('fls', data=fls_out)
    f_new.create_dataset('wls', data=wls_out)
    f_new.create_dataset('sigmas', data=sig_out)
    f_new.create_dataset('masks', data=msk_out)
    print("{:03d}: {:.0f}  -  {:.0f}   {}".format(i, wls_out[0], wls_out[-1], out_name))
    f_new.close()

000: 10065  -  24949   ../data/reduced/2M_J0050.hdf5
001: 10114  -  24985   ../data/reduced/2M_J0415.hdf5
002: 10065  -  24949   ../data/reduced/2M_J0727.hdf5
003: 10051  -  24940   ../data/reduced/2M_J0729.hdf5
004: 10052  -  24940   ../data/reduced/2M_J0939.hdf5
005: 10032  -  24926   ../data/reduced/2M_J1114.hdf5
006: 10080  -  24960   ../data/reduced/2M_J1217.hdf5
007: 10103  -  24977   ../data/reduced/2M_J1553.hdf5
008: 10025  -  24921   ../data/reduced/Gl570D.hdf5
009: 10055  -  24942   ../data/reduced/HD3651B.hdf5
010: 10037  -  24929   ../data/reduced/SDSS_1416b.hdf5


### The end!